# TODO
* [ ] Correrlo con `max_iter = 5_000`
* [ ] Hacer la versión Batch
* [ ] Hacer la versión proyectada

Notebook que sirve de ejemplo para calcular el baricentro de un conjunto de datos de imágenes.

# Configuraciones iniciales

## Constantes

In [ ]:
NOTEBOOK = 3
CLEAN_LOGS = True  # If you want to clean the logs directory
SAVE_FIGS = True  # If you want to save the figures.

REPORT_EVERY = 50  # To report at the logger
MAX_ITER = 5_000  # MAx number of iterations for the SGDW
BATCH_SIZE = 5
PROJ_EVERY = 1

# MAX_ITER = 50; REPORT_EVERY = 5  # Descomentar para debuguear

In [ ]:
import torch
import numpy as np
import random

SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.backends.cudnn.deterministic = True

In [ ]:
from pathlib import Path
from icecream import ic


DS_NAME = "data"

CURR_PATH = Path().absolute()
ic(CURR_PATH)
BASE_PATH = CURR_PATH.parent.parent
ic(BASE_PATH)
DATA_PATH = BASE_PATH / "data"
ic(DATA_PATH)
WGAN_PATH = BASE_PATH / "wgan_gp"
ic(WGAN_PATH)
NETS_PATH = WGAN_PATH / "networks"
ic(NETS_PATH)
IMGS_PATH = CURR_PATH / "imgs" / f"notebook-{NOTEBOOK:02d}"
IMGS_PATH.mkdir(parents=True, exist_ok=True)
ic(IMGS_PATH)

## Importaciones generales

In [ ]:
from bwb import utils
from bwb.sgdw import sgdw
from bwb.distributions import *
import torch
import bwb.utils.plotters as plotters
import bwb.sgdw.plotters as sgdw_plotters

## Configuraciones 

In [ ]:
from bwb.config import conf

conf.use_gpu()
conf.use_single_precision()
conf.set_eps(1e-16)
conf

## Configuración del Logger

In [ ]:
import time
from pathlib import Path


# Create the logs directory
LOG_PATH = (
    Path("logs")
    / f"notebook-{NOTEBOOK:02d}_{time.strftime('%Y%m%d_%H%M%S')}.log"
)
if not LOG_PATH.parent.exists():
    LOG_PATH.parent.mkdir()

# Clean the logs
if CLEAN_LOGS:
    for log_file in Path("logs").glob(f"notebook-{NOTEBOOK:02d}*.log"):
        log_file.unlink()

In [ ]:
import logging
from bwb._logging import log_config


# Remove the handlers
log_config.remove_all_handlers()

# Define and add FileHandler
fh = logging.FileHandler(LOG_PATH)
log_config.set_default_formatter(fh)
log_config.add_handler(fh)


_log = log_config.get_logger("notebook")
log_config.set_level(level=logging.DEBUG)
log_config.set_level(
    level=logging.INFO, name="bwb.distributions.discrete_distribution"
)

Esta celda es para configurar la información mostrada en el logger

In [ ]:
# Set the default options for the report
INCLUDE_OPTIONS: sgdw.ReportOptions = {
    "dt": False,
    "dt_per_iter": True,
    "iter": True,
    "step_schd": True,
    "total_time": True,
    "w_dist": False,
}

sgdw.Report.INCLUDE_OPTIONS = INCLUDE_OPTIONS

## Obtención del dataset

In [ ]:
# You can use the wrapper to transform the usual DataSet into a model set
from bwb.distributions.models import ModelDataset
import quick_torch as qt
import torchvision.transforms.v2 as T

transform_ds = T.Compose([
    T.Resize((32, 32)),
    T.ToImage(),
    T.ToDtype(torch.float32, scale=True),
    T.Lambda(lambda x: x.squeeze()),
])


def get_ds(file_path, transform=transform_ds):
    ic(file_path)
    categories = [qt.Category.FACE]
    dataset_ = qt.QuickDraw(
        root=DATA_PATH,
        categories=categories,
        transform=transform,
        download=True,
        recognized=True,
    )
    path_dataset = Path(file_path)
    dataset_.data = np.load(path_dataset).reshape(-1, 28, 28)
    dataset_.targets = np.ones(len(dataset_.data), dtype=int)
    dataset = dataset_.get_train_data()
    ic(len(dataset))

    return ModelDataset(dataset)


DS_PATH = WGAN_PATH / "dataset" / "cleaned" / f"{DS_NAME}.npy"
ds_models = get_ds(DS_PATH)

ds_models.get(0)

In [ ]:
ds_models

## Obtener GAN

De la misma manera, se puede definir un muestreador de distribuciones utilizando una GAN. Para ello, empezamos definiendo las redes neuronales a utilizar

In [ ]:
from wgan_gp.wgan_gp_vae.model_resnet import (
    Generator,
    Encoder,
    LatentDistribution,
)
import torch
from wgan_gp.wgan_gp_vae.utils import load_checkpoint


device = conf.device

NOISE = "norm"
LATENT_DIM = 128
CHANNELS_IMG = 1
NUM_FILTERS = [256, 128, 64, 32]

noise_sampler = LatentDistribution(NOISE, LATENT_DIM, device, conf.dtype)

DS_NAME = "data"

to_conf = dict(device=conf.device, dtype=conf.dtype, non_blocking=True)
G = Generator(LATENT_DIM, CHANNELS_IMG, latent_distr=NOISE).to(**to_conf)
E = Encoder(LATENT_DIM, CHANNELS_IMG).to(**to_conf)

FACE_PATH = NETS_PATH / f"cleaned_{DS_NAME}_zDim{LATENT_DIM}_{NOISE}_bs_128"
ic(FACE_PATH)

load_checkpoint(G, FACE_PATH, "generator", device)
load_checkpoint(E, FACE_PATH, "encoder", device)

G.eval()
E.eval()
print()

In [ ]:
noise_sampler

In [ ]:
from bwb.distributions import DistributionDraw
from torchvision import disable_beta_transforms_warning

disable_beta_transforms_warning()

import torchvision.transforms.v2 as T


z = noise_sampler(1)
m = G(z)

transform_in = T.Compose([
    T.Lambda(lambda x: x / torch.max(x)),
    T.ToPILImage(),
    T.Resize(32),
    T.ToImage(),
    T.ConvertImageDtype(conf.dtype),
    T.Normalize((0.5,), (0.5,)),
])

transform_out_ = T.Compose([
    T.ToDtype(conf.dtype),
    T.Lambda(lambda x: x.squeeze()),
    T.Lambda(lambda x: x - torch.min(x)),
    T.Lambda(lambda x: x / torch.sum(x)),
])

transform_out = T.Compose([
    transform_out_,
    T.Lambda(lambda x: DistributionDraw.from_grayscale_weights(x)),
])


out: DistributionDraw = transform_out(m)
print(out.dtype)
out

## Definir proyector

In [ ]:
from wgan_gp.wgan_gp_vae.utils import ProjectorOnManifold
import torchvision.transforms.v2 as T

transform_in_proj = T.Compose([
    # From pdf to grayscale
    T.Lambda(lambda x: x / torch.max(x)),
    T.ToPILImage(),
    T.Resize((32, 32)),
    T.ToImage(),
    T.ToDtype(conf.dtype, scale=True),
    T.Normalize(
        [0.5 for _ in range(1)],
        [0.5 for _ in range(1)],
    ),
])

transform_out_proj = T.Compose([
    # Ensure the range is in [0, 1]
    T.Lambda(lambda x: x - torch.min(x)),
    T.Lambda(lambda x: x / torch.max(x)),
    T.Lambda(lambda x: x / torch.sum(x)),
    T.Lambda(lambda x: x.squeeze(0)),
])

proj = ProjectorOnManifold(
    E,
    G,
    transform_in=transform_in_proj,
    transform_out=transform_out_proj,
)

## Definir $\gamma_k$

Aquí se utiliza una función de la forma
\begin{equation*}
    \gamma_k = \frac{a}{(b^{1/c} + k)^c}
\end{equation*}

Con $a > 0$, $b \geq 0$ y $0.5 < c \leq 1$

La idea es que cuando $k=0$, $\gamma_0 = \frac{a}{b}$ es la proporción entre $a$ y $b$, permitiendo ajustar el valor inicial.

In [ ]:
from bwb.sgdw.utils import step_scheduler

window = 5


def test_gamma(gamma):

    for t in range(window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 50
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 100
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 300
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 500
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 1_000
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 3_000
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()

    init = 5_000
    for t in range(init, init + window):
        print(f"{t = :_}; {gamma(t) = :.2%}")
    print()


_a = 3
_eps = 1e-3
params = dict(a=_a, b=_a + 1e-2, c=0.5 + _eps)
# params = dict(a=1, b=1, c=1)

gamma = step_scheduler(**params)

test_gamma(step_scheduler(**params))

# Baricentro de imágenes

Para obtener el baricentro de un conjunto de imágenes, es necesario utilizar la clase `UniformDiscreteSampler` y fijarla con una clase que tenga los siguientes métodos:
* `get(i) -> Distribution` que retorne la distribución $i$-ésima.
* `__len__() -> int` que retorne el tamaño del dataset.

In [ ]:
distr_sampler = UniformDiscreteSampler[DistributionDraw]().fit(models=ds_models)
repr(distr_sampler.draw())

Luego definimos el algoritmo a utilizar. En este caso, utilizaremos `DebiesedDistributionDrawSGDW` que realiza los transportes utilizando un método convolución debiesed. 

In [ ]:
dist_draw_sgdw = sgdw.DebiesedDistributionDrawSGDW(
    projector=proj,
    proj_every=PROJ_EVERY,
    distr_sampler=distr_sampler,
    step_scheduler=step_scheduler(**params),
    batch_size=BATCH_SIZE,
    max_iter=MAX_ITER,
    report_every=REPORT_EVERY,
)
dist_draw_sgdw.det_params, dist_draw_sgdw.hist

Definimos una clase para comparar las imágenes de las muestras con la iteración del algoritmo.

In [ ]:
from bwb.sgdw.plotters import PlotterComparison

plotter_comp = plotter_comp_ds = PlotterComparison(
    dist_draw_sgdw, n_cols=12, n_rows=1, cmap="binary_r"
)

In [ ]:
_log.info(
    f"Running SGD-Wasserstein with '{DS_NAME}' DS barycenter projected and"
    " batched"
)
bar = plotter_comp.run(include_dict=dict(total_time=True))
dist_draw_sgdw.iter_params

Obtenemos una visualización de las primeras imágenes.

In [ ]:
_ = plotter_comp.plot(0)

In [ ]:
_ = plotter_comp_ds.plot(1625)

Obtenemos una visualización de las últimas imágenes.

In [ ]:
_ = plotter_comp.plot()

# Baricentro de la GAN

## Definir el algoritmo

In [ ]:
distr_sampler = GeneratorDistribSampler()
distr_sampler.fit(
    generator=G, noise_sampler=noise_sampler, transform_out=transform_out_
)
distr_sampler.draw()

In [ ]:
dist_draw_sgdw = sgdw.DebiesedDistributionDrawSGDW(
    projector=proj,
    proj_every=PROJ_EVERY,
    distr_sampler=distr_sampler,
    step_scheduler=step_scheduler(**params),
    batch_size=BATCH_SIZE,
    max_iter=MAX_ITER,
    report_every=REPORT_EVERY,
)
dist_draw_sgdw.det_params, dist_draw_sgdw.hist

Definimos una clase para comparar las imágenes de las muestras con la iteración del algoritmo.

In [ ]:
from bwb.sgdw.plotters import PlotterComparison

plotter_comp = plotter_comp_gan = PlotterComparison(
    dist_draw_sgdw, n_cols=12, n_rows=1, cmap="binary_r"
)

In [ ]:
_log.info(
    f"Running SGD-Wasserstein with '{DS_NAME}' GAN barycenter projected and"
    " batched"
)
bar = plotter_comp.run(include_dict=dict(total_time=True))
dist_draw_sgdw.iter_params

Obtenemos una visualización de las primeras imágenes.

In [ ]:
_ = plotter_comp.plot(0)

In [ ]:
_ = plotter_comp.plot(1500)

Obtenemos una visualización de las últimas imágenes.

In [ ]:
_ = plotter_comp.plot()